Imports & Dataset

In [1]:
pip install tensorflow deap numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\aswat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import random
import numpy as np
import tensorflow as tf

from deap import base, creator, tools, algorithms

# Load MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784) / 255.0
x_test  = x_test.reshape(-1, 784) / 255.0


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Fitness & Individual Definition (NSGA-II)

In [ ]:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)


Chromosome Generator

In [ ]:
def create_individual():
    n_layers = random.randint(1, 3)
    units = [random.randint(16, 128) for _ in range(3)]
    lr = random.uniform(1e-4, 1e-2)
    return creator.Individual([n_layers] + units + [lr])


Decode Chromosome → TensorFlow Model

In [ ]:
def build_model(individual):
    n_layers = individual[0]
    units = individual[1:1+n_layers]
    lr = individual[-1]

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(784,)))

    for u in units:
        model.add(tf.keras.layers.Dense(u, activation="relu"))

    model.add(tf.keras.layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


Fitness Evaluation

In [9]:
def evaluate(individual):
    model = build_model(individual)

    model.fit(
        x_train, y_train,
        epochs=2,
        batch_size=128,
        verbose=0
    )

    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    params = model.count_params()

    return loss, params


Register NSGA-II Operators

In [7]:
toolbox = base.Toolbox()

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=10, indpb=0.2)
toolbox.register("select", tools.selNSGA2)


Run Evolution

In [8]:
population = toolbox.population(n=10)
generations = 5

for gen in range(generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.9, mutpb=0.1)

    for ind in offspring:
        ind.fitness.values = toolbox.evaluate(ind)

    population = toolbox.select(population + offspring, k=len(population))
    print(f"Generation {gen} completed")


Generation 0 completed
Generation 1 completed
Generation 2 completed
Generation 3 completed


ValueError: Cannot convert '(784, 77.95840357465151)' to a shape. Found invalid entry '77.95840357465151' of type '<class 'float'>'. 

Result (Pareto-Optimal Networks)

In [ ]:
pareto_front = tools.sortNondominated(population, k=len(population), first_front_only=True)[0]

for ind in pareto_front:
    print("Architecture:", ind, "Fitness:", ind.fitness.values)
